<a href="https://colab.research.google.com/github/Tensor-Reloaded/Python-IA/blob/main/04-Joi/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatBoost

## $$1.\ Pregătirea\ Datelor$$
### 1.1 Instalarea CatBoost
Dacă nu ai instalat deja CatBoost, o poți face rulând comanda '!pip install catboost'.  
  
De asemenea, ar trebui să instalezi pachetul ipywidgets și să rulezi o comandă specială înainte de a porni Jupyter Notebook pentru a putea desena grafice.

In [1]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

### 1.2 Încărcarea Datelor
Datele pentru acest tutorial pot fi obținute de pe [această pagină](https://www.kaggle.com/c/titanic/data) (va trebui să vă creați un cont Kaggle sau să vă conectați cu Facebook sau Google+) sau puteți folosi catboost.datasets așa cum este în codul de mai jos.

In [3]:
from catboost.datasets import titanic
import numpy as np

train_df, test_df = titanic()

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 1.3 Pregătirea Caracteristicilor
În primul rând, să verificăm câte valori absente avem:

In [4]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

,0
Age,177
Cabin,687
Embarked,2


După cum putem observa, **`Age`**, **`Cabin`** și **`Embarked`** au într-adevăr unele valori lipsă, așa că haideți să le completăm cu un număr care să fie în afara distribuțiilor lor - astfel încât modelul să poată distinge ușor între ele și să le ia în considerare:

In [5]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Acum să separăm variabilele de caracteristici și eticheta:

In [6]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

Atenție, caracteristicile noastre sunt de tipuri diferite - unele dintre ele sunt numerice, altele sunt categorice și unele sunt chiar șiruri de caractere, care în mod normal ar trebui gestionate într-un mod specific (de exemplu, codificate cu reprezentarea bag-of-words). Dar, în cazul nostru, putem trata aceste caracteristici de tip șiruri de caractere la fel ca pe cele categorice - toată munca grea este realizată în interiorul CatBoost. Cât de tare e asta? :)

In [7]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != float)[0]

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


### 1.4 Împărțirea Datelor
Să împărțim datele de antrenament în seturi de antrenament și validare.

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

## $$2.\ CatBoost\$$

Importuri necesare.

In [9]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

### 2.1 Antrenarea Modelului
Acum să creăm modelul propriu-zis. Vom folosi parametrii prestabiliți, deoarece aceștia oferă aproape întotdeauna un punct de plecare _foarte_ bun. Singurul lucru pe care dorim să-l specificăm aici este parametrul `custom_loss`, deoarece acesta ne va permite să vedem ce se întâmplă în ceea ce privește metricile competiției - acuratețea, precum și să monitorizăm logloss-ul, deoarece va fi mai lin pe un set de date de această dimensiune.

In [10]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [11]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 2.2 Validarea Încrucișată a Modelului

Este bine să validezi modelul tău, dar să îl validezi încrucișat este chiar mai bine. Și, de asemenea, cu grafice! Așadar, fără alte cuvinte:

In [12]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [13]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.83±0.02 on step 355


In [14]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8294051627384961


După cum putem observa, estimarea noastră inițială a performanței pe un singur set de validare a fost prea optimistă - de aceea validarea încrucișată este atât de importantă!

### 2.3 Aplicarea Modelului
Tot ce trebuie să faci pentru a obține predicțiile este

In [15]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0 0 0 0 1 0 1 0 1 0]
[[0.85473931 0.14526069]
 [0.76313031 0.23686969]
 [0.88972889 0.11027111]
 [0.87876173 0.12123827]
 [0.3611047  0.6388953 ]
 [0.90513381 0.09486619]
 [0.33434185 0.66565815]
 [0.78468564 0.21531436]
 [0.39429048 0.60570952]
 [0.94047549 0.05952451]]


Dar haideți să încercăm să obținem predicții mai bune, iar caracteristicile CatBoost ne ajută în acest sens.

## $$3.\ CatBoost\ Features$$
Probabil ați observat că în etapa de creare a modelului am specificat nu doar `custom_loss`, ci și parametrul `random_seed`. Acest lucru a fost făcut pentru ca acest notebook să fie reproductibil - în mod implicit, CatBoost alege o valoare aleatorie pentru seed:

In [16]:
model_without_seed = CatBoostClassifier(iterations=10, logging_level='Silent')
model_without_seed.fit(X, y, cat_features=categorical_features_indices)

print('Random seed assigned for this model: {}'.format(model_without_seed.random_seed_))

Random seed assigned for this model: 0


Să definim câțiva parametri și să creăm un `Pool` pentru mai multă comoditate. Acesta stochează toate informațiile despre setul de date (caracteristici, etichete, indicii caracteristicilor categorice, greutăți și multe altele).

In [17]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

### 3.1 Utilizarea celui mai bun model
Dacă ai un set de validare, este întotdeauna mai bine să folosești parametrul `use_best_model` în timpul antrenamentului. În mod implicit, acest parametru este activat. Dacă este activat, ansamblul de arbori rezultat se va reduce la cea mai bună iterație.

In [18]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

Simple model validation accuracy: 0.7982

Best model validation accuracy: 0.8251


### 3.2 Early Stopping
Dacă ai un set de validare, este întotdeauna mai ușor și mai bine să folosești oprirea timpurie. Această caracteristică este similară cu cea anterioară, dar pe lângă îmbunătățirea calității, economisește și timp.

In [19]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

CPU times: user 5.94 s, sys: 573 ms, total: 6.51 s
Wall time: 4.87 s


In [20]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

CPU times: user 841 ms, sys: 78.8 ms, total: 920 ms
Wall time: 647 ms


In [21]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Simple model tree count: 500
Simple model validation accuracy: 0.7982

Early-stopped model tree count: 82
Early-stopped model validation accuracy: 0.8072


Astfel, obținem o calitate mai bună într-un timp mai scurt.

Deși, așa cum s-a arătat anterior, schema simplă de validare nu descrie cu precizie scorul modelului în afara setului de antrenament (poate fi părtinitoare din cauza împărțirii setului de date), este totuși util să urmărești dinamica îmbunătățirii modelului - și, după cum putem vedea din acest exemplu, este foarte bine să oprim procesul de boosting mai devreme (înainte de a apărea supraînvățarea).

### 3.3 Utilizarea Baseline-ului
Este posibil să folosim rezultatele pre-antrenamentului (baseline) pentru antrenare.

In [22]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# Get baseline (only with prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type='RawFormulaVal')
# Fit new model
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline);

### 3.4 Suport pentru Snapshot
CatBoost suportă snapshot-uri. Le poți folosi pentru a relua antrenamentul după o întrerupere sau pentru a începe antrenamentul cu rezultatele anterioare.

In [23]:
params_with_snapshot = params.copy()
params_with_snapshot.update({
    'iterations': 5,
    'learning_rate': 0.5,
    'logging_level': 'Verbose'
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)
params_with_snapshot.update({
    'iterations': 10,
    'learning_rate': 0.1,
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)


bestTest = 0.802690583
bestIteration = 4


bestTest = 0.802690583
bestIteration = 4



### 3.5 Funcție Obiectiv Definită de Utilizator
Este posibil să creezi propria funcție obiectiv. Haideți să creăm o funcție obiectiv pentru logloss.

In [24]:
# for performance reasons it is better to install `numba` package for working with user defined functions
!pip install numba

In [25]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [26]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42,
    loss_function=LoglossObjective(),
    eval_metric=metrics.Logloss()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

0:	learn: 0.6827074	total: 827ms	remaining: 7.45s
1:	learn: 0.6723302	total: 830ms	remaining: 3.32s
2:	learn: 0.6619449	total: 832ms	remaining: 1.94s
3:	learn: 0.6521466	total: 835ms	remaining: 1.25s
4:	learn: 0.6435227	total: 837ms	remaining: 837ms
5:	learn: 0.6353848	total: 840ms	remaining: 560ms
6:	learn: 0.6277210	total: 843ms	remaining: 361ms
7:	learn: 0.6210282	total: 845ms	remaining: 211ms
8:	learn: 0.6141958	total: 847ms	remaining: 94.1ms
9:	learn: 0.6073236	total: 850ms	remaining: 0us


### 3.6 Funcție Metrică Definită de Utilizator
De asemenea, este posibil să creezi propria funcție metrică. Haideți să creăm o funcție metrică pentru logloss.

In [27]:
class LoglossMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += -w * (target[i] * approx[i] - np.log(1 + np.exp(approx[i])))

        return error_sum, weight_sum

In [28]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42,
    loss_function=metrics.Logloss(),
    eval_metric=LoglossMetric()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 0.5521578	total: 299ms	remaining: 2.69s
1:	learn: 0.4885686	total: 309ms	remaining: 1.23s
2:	learn: 0.4607664	total: 313ms	remaining: 730ms
3:	learn: 0.4418819	total: 319ms	remaining: 479ms
4:	learn: 0.4278162	total: 324ms	remaining: 324ms
5:	learn: 0.4151036	total: 330ms	remaining: 220ms
6:	learn: 0.4099336	total: 334ms	remaining: 143ms
7:	learn: 0.4095363	total: 339ms	remaining: 84.7ms
8:	learn: 0.4032867	total: 344ms	remaining: 38.2ms
9:	learn: 0.3929586	total: 350ms	remaining: 0us


### 3.7 Predicție Etapizată
Modelul CatBoost are metoda `staged_predict`. Aceasta îți permite să obții iterativ predicții pentru un interval dat de arbori.

In [29]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
ntree_start, ntree_end, eval_period = 3, 9, 2
predictions_iterator = model.staged_predict(validate_pool, 'Probability', ntree_start, ntree_end, eval_period)
for preds, tree_count in zip(predictions_iterator, range(ntree_start, ntree_end, eval_period)):
    print('First class probabilities using the first {} trees: {}'.format(tree_count, preds[:5, 1]))

First class probabilities using the first 3 trees: [0.53597869 0.41039128 0.42057479 0.64281031 0.46576685]
First class probabilities using the first 5 trees: [0.63722688 0.42492029 0.46209302 0.70926021 0.44280772]
First class probabilities using the first 7 trees: [0.66964764 0.42409144 0.46124982 0.76101033 0.47205986]


### 3.8 Importanța Caracteristicilor
Uneori este foarte important să înțelegem care caracteristică a avut cea mai mare contribuție la rezultatul final. Pentru acest lucru, modelul CatBoost are o metodă `get_feature_importance`.

In [30]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Sex: 59.004092014268586
Pclass: 16.340887169747035
Ticket: 6.028107169932204
Cabin: 3.8347242202560192
Fare: 3.712969667934384
Age: 3.484451204182482
Parch: 3.378089740355865
Embarked: 2.3139994072899555
SibSp: 1.9026794060334504
PassengerId: 0.0
Name: 0.0


Acest lucru arată că caracteristicile **`Sex`** și **`Pclass`** au avut cea mai mare influență asupra rezultatului.

### 3.9 Evaluarea Metricilor
CatBoost are o metodă `eval_metrics` care permite calcularea unor metrici date pe un set de date specific. Și, bineînțeles, să le și afișeze :)

In [31]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
eval_metrics = model.eval_metrics(validate_pool, [metrics.AUC()], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [32]:
print(eval_metrics['AUC'][:6])

[0.8627368774106994, 0.8623176253563642, 0.8602213650846889, 0.8514170719436525, 0.8495723629045783, 0.8569092738554419]


### 3.10 Compararea Proceselor de Învățare
Poți, de asemenea, să compari procesul de învățare al diferitelor modele pe un singur grafic.

In [33]:
model1 = CatBoostClassifier(iterations=100, depth=1, train_dir='model_depth_1/', logging_level='Silent')
model1.fit(train_pool, eval_set=validate_pool)
model2 = CatBoostClassifier(iterations=100, depth=5, train_dir='model_depth_5/', logging_level='Silent')
model2.fit(train_pool, eval_set=validate_pool);

In [34]:
from catboost import MetricVisualizer
widget = MetricVisualizer(['model_depth_1', 'model_depth_5'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 3.11 Salvarea Modelului
Este întotdeauna foarte util să poți salva modelul pe disc (mai ales dacă antrenamentul a durat ceva timp).

In [35]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
model = CatBoostClassifier()
model.load_model('catboost_model.dump');

# $$4.\ Parameters\ Tuning$$
Deși poți selecta întotdeauna numărul optim de iterații (pași de boosting) prin validare încrucișată și grafice ale curbei de învățare, este de asemenea important să experimentezi cu anumiți parametri ai modelului, iar noi dorim să acordăm o atenție specială parametrilor `l2_leaf_reg` și `learning_rate`.

În această secțiune, vom selecta acești parametri folosind pachetul **`hyperopt`**.

In [36]:
!pip install hyperopt

In [37]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric=metrics.Accuracy(),
        random_seed=42,
        verbose=False,
        loss_function=metrics.Logloss(),
    )

    cv_data = cv(
        Pool(X, y, cat_features=categorical_features_indices),
        model.get_params(),
        logging_level='Silent',
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])

    return 1 - best_accuracy # as hyperopt minimises

In [38]:
from numpy.random import default_rng
import hyperopt

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()
rng = default_rng(123)

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=rng
)

print(best)


100%|██████████| 50/50 [11:13<00:00, 13.46s/trial, best loss: 0.16835016835016825]
{'l2_leaf_reg': 3.0, 'learning_rate': 0.31169226963487207}


Acum să obținem toate datele de validare încrucișată (cv) cu cei mai buni parametri:

In [39]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric=metrics.Accuracy(),
    random_seed=42,
    verbose=False,
    loss_function=metrics.Logloss(),
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

Training on fold [0/3]

bestTest = 0.835016835
bestIteration = 55

Training on fold [1/3]

bestTest = 0.8451178451
bestIteration = 58

Training on fold [2/3]

bestTest = 0.8249158249
bestIteration = 41



In [40]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8316498316498318


Rețineți că, cu parametrii prestabiliți, scorul nostru de validare încrucișată (cv) a fost 0.8283 și, prin urmare, avem o anumită îmbunătățire.

### Crearea Predictiei
Acum vom re-antrena modelul nostru ajustat pe toate datele de antrenament pe care le avem.

In [41]:
model.fit(X, y, cat_features=categorical_features_indices)

In [42]:
import pandas as pd
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model.predict(X_test)

In [43]:
submisstion.to_csv('submission.csv', index=False)

In [44]:
from base64 import b64decode
from IPython.display import HTML, Image
from google.colab.output import eval_js
import urllib.request
board_html = urllib.request.urlopen('https://gist.githubusercontent.com/karim23657/5ad5e067c1684dbc76c93bd88bf6fa53/raw/2ef57f881bc700c2c346bd6c7a7f2d5364b21048/drawing%2520board.html').read().decode('utf-8')
def draw(filename='drawing.png'):
  display(HTML(board_html))
  data = eval_js('triggerImageToServer')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  display(Image('drawing.png'))

In [ ]:
draw('drawing.png')